In [1]:
import ngeodesic

ImportError: cannot import name 'stock_parse' from 'ngeodesic.core.parser' (/Users/ian_moore/repos/ngeodesic/python/ngeodesic/core/parser.py)

In [ ]:
import numpy as np
from ngeodesic.core.denoise import TemporalDenoiser
from ngeodesic.core.parser import geodesic_parse_report  # Stage-10/11 parser
from ngeodesic.core.matched_filter import half_sine_proto